# Load model

In [4]:
import torch.nn as nn
import torch
class VGG(nn.Module):
    def __init__(self, features, DROPOUT=0.5, NUM_CLASSES=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(DROPOUT),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(DROPOUT),
            nn.Linear(4096, 2048),
            nn.ReLU(True),
            nn.Dropout(DROPOUT),
            nn.Linear(2048, 1024),
            nn.ReLU(True),
            nn.Dropout(DROPOUT),
            nn.Linear(1024, 512),
            nn.ReLU(True),
            nn.Dropout(DROPOUT),
            nn.Linear(512, NUM_CLASSES)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNom2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

vgg_model = VGG(make_layers(cfg), DROPOUT=0.5, NUM_CLASSES=1, init_weights=False)

checkpoint = torch.load(r"D:\Local\Source\python\semester_6\face_attribute\checkpoints\vgg13\best_checkpoint.pt")
vgg_model.load_state_dict(checkpoint['model_state_dict'])

RuntimeError: Error(s) in loading state_dict for VGG:
	Missing key(s) in state_dict: "features.14.weight", "features.14.bias", "features.16.weight", "features.16.bias", "features.19.weight", "features.19.bias", "features.21.weight", "features.21.bias", "features.23.weight", "features.23.bias", "features.25.weight", "features.25.bias", "features.28.weight", "features.28.bias", "features.30.weight", "features.30.bias", "features.32.weight", "features.32.bias", "features.34.weight", "features.34.bias". 
	Unexpected key(s) in state_dict: "features.15.weight", "features.15.bias", "features.17.weight", "features.17.bias", "features.20.weight", "features.20.bias", "features.22.weight", "features.22.bias". 

In [ ]:
vgg_model.eval()

# VN Express set evaluate

In [13]:
from PIL import Image
import os
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from torchvision.transforms import transforms
from torchvision.transforms import InterpolationMode

In [14]:
import os
import pandas as pd

folder = '/kaggle/input/test-data-version-0-4/Test'
data = {'image_path': [], 'label': []}

for filename in os.listdir(folder):
    image_path = os.path.join(folder, filename)
    
    if 'Female' in filename:
        label = 0
    elif 'Male' in filename:
        label = 1
    else:
        continue

    data['image_path'].append(image_path)
    data['label'].append(label)

df = pd.DataFrame(data)

In [15]:
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader
from torch.utils.data import Dataset  

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image_path']
        label = self.data.iloc[idx]['label']

        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

test_set = CustomDataset(dataframe=df,
                            transform=v2.Compose([
                                v2.Resize(size=(224, 224), interpolation=InterpolationMode.BICUBIC),
                                v2.PILToTensor(),
                                v2.ToDtype(torch.float32, scale=True),
                                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                            ]))

test_set = DataLoader(dataset=test_set,
                      batch_size=32,
                      shuffle=True,
                      num_workers=2,
                      pin_memory=True
                      )

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

# Assuming you have already created the DataLoader 'test_set'
# Get the first batch
batch_images, batch_labels = next(iter(test_set))

# Function to unnormalize the images
def unnormalize(img):
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    img = img * std[:, None, None] + mean[:, None, None]
    return img

# Function to show images
def show_images(images, labels):
    fig, axes = plt.subplots(4, 8, figsize=(15, 8))
    for i, ax in enumerate(axes.flat):
        image = unnormalize(images[i]).permute(1, 2, 0).numpy()
        ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"Label: {labels[i]}")

# Show the images in the first batch
show_images(batch_images, batch_labels)
plt.show()

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from tqdm import tqdm

thresholds = torch.arange(0.1, 1.0, 0.1)
best_f1_score = 0 
vgg_model.to('cuda')

for threshold in thresholds:
    predictions = []
    labels = []

    with torch.no_grad():
        for index, batch in tqdm(enumerate(test_set), total=len(test_set), desc="Test"):
            imgs, ground_truths = batch[0].type(torch.FloatTensor), batch[1].type(torch.FloatTensor)

            imgs = imgs.to("cuda")
            ground_truths = ground_truths.to("cuda").detach().cpu().numpy().tolist()
            preds = torch.nn.functional.sigmoid(vgg_model(imgs)).squeeze().detach().cpu().numpy().tolist()
            
            for pred, ground_truth in zip(preds, ground_truths):
                #female 0, male 1
                label= 1 if pred>threshold else 0
                predictions.append(label)
                labels.append(ground_truth)

    conf_matrix = confusion_matrix(labels, predictions)
    macro_average_recall = recall_score(labels, predictions, average='macro', zero_division=1)
    macro_average_precision = precision_score(labels, predictions, average='macro', zero_division=1)
    macro_average_f1_score = 2 * (macro_average_precision * macro_average_recall) / (macro_average_precision + macro_average_recall)
    print(f'Threshold: {round(threshold.item(),1)} Precision: {macro_average_precision} Recall: {macro_average_recall} F1-Score: {macro_average_f1_score}')

In [ ]:
predictions = []
labels = []

with torch.no_grad():
    for index, batch in tqdm(enumerate(test_set), total=len(test_set), desc="Test"):
        imgs, ground_truths = batch[0].type(torch.FloatTensor), batch[1].type(torch.FloatTensor)

        imgs = imgs.to("cuda")
        ground_truths = ground_truths.to("cuda").detach().cpu().numpy().tolist()
        preds = torch.nn.functional.sigmoid(vgg_model(imgs)).squeeze().detach().cpu().numpy().tolist()

        for pred, ground_truth in zip(preds, ground_truths):
            #female 0, male 1
            label= 1 if pred>0.9 else 0
            predictions.append(label)
            labels.append(ground_truth)

conf_matrix = confusion_matrix(labels, predictions)
macro_average_recall = recall_score(labels, predictions, average='macro', zero_division=1)
macro_average_precision = precision_score(labels, predictions, average='macro', zero_division=1)
macro_average_f1_score = 2 * (macro_average_precision * macro_average_recall) / (macro_average_precision + macro_average_recall)
print(f'Threshold: {round(threshold.item(),1)} Precision: {macro_average_precision} Recall: {macro_average_recall} F1-Score: {macro_average_f1_score}')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("Confusion Matrix:")
print(conf_matrix)

fig=plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

fig.savefig('/kaggle/working/CFS')

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(labels, predictions)
roc_auc = auc(fpr, tpr)

fig1=plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='AUC = {:.2f}'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('AUC-ROC Curve')
plt.legend()
plt.show()
fig1.savefig('/kaggle/working/AUC')
precision, recall, _ = precision_recall_curve(labels, predictions)

fig2=plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', lw=2, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()
fig2.savefig('/kaggle/working/PRE-RE')